<a href="https://colab.research.google.com/github/Akash-Aranganathan/AF2RAVE-toolset/blob/main/AF2_RAVE_Block1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Installations of dependencies

Run each cell carefully (do not give `Run all` )

Runtime for installations `~11mins`

In [ ]:
#@title 1.1) Install condacolab
#@markdown After running this cell wait for the kernel to restart (~1min)

#@markdown Then, start running the rest of the blocks
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

#from time import sleep
#sleep(10)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:20
🔁 Restarting kernel...


In [ ]:
#@title 1.2) Install ColabFold and Plumed
# setup device
import time
t1 = time.perf_counter()
from IPython.display import clear_output
!mkdir struct_gen
%cd /content/struct_gen/.
clear_output()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
import tensorflow as tf
import jax

try:
  # check if TPU is available
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()
  print('Running on TPU')
  DEVICE = "tpu"
except:
  if jax.local_devices()[0].platform == 'cpu':
    print("WARNING: no GPU detected, will be using CPU")
    DEVICE = "cpu"
  else:
    print('Running on GPU')
    DEVICE = "gpu"
    # disable GPU on tensorflow
    tf.config.set_visible_devices([], 'GPU')

from IPython.utils import io
import subprocess
import tqdm.notebook


install_jackhmmer = False      #not required

#AF2 repo from deepmind
GIT_REPO = 'https://github.com/deepmind/alphafold'
#AF2 params
SOURCE_URL = 'https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar'

PARAMS_DIR = './alphafold/data/params'
PARAMS_PATH = os.path.join(PARAMS_DIR, os.path.basename(SOURCE_URL))

TMP_DIR = "tmp"
os.makedirs(TMP_DIR, exist_ok=True)

#tqdm specification
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} '
# if not already installed
total = 75
with tqdm.notebook.tqdm(total=total, bar_format=TQDM_BAR_FORMAT) as pbar:
  if not os.path.isdir("alphafold"):
    # download alphafold code and clone colabfold repo
    os.system(f"git clone {GIT_REPO} alphafold; cd alphafold; git checkout 1d43aaff941c84dc56311076b58795797e49107b")
    os.system(f"git clone https://github.com/sokrypton/ColabFold.git")

    # apply patches
    os.system(f"patch -u alphafold/alphafold/model/model.py -i ColabFold/beta/model.patch")
    os.system(f"patch -u alphafold/alphafold/model/mapping.py -i ColabFold/beta/mapping.patch")
    os.system(f"patch -u alphafold/alphafold/model/modules.py -i ColabFold/beta/modules.patch")
    os.system(f"patch -u alphafold/alphafold/model/folding.py -i ColabFold/beta/folding.patch")
    os.system(f"patch -u alphafold/alphafold/model/config.py -i ColabFold/beta/config.patch")
    # apply multi-chain patch from Lim Heo @huhlim
    os.system(f"patch -u alphafold/alphafold/common/protein.py -i ColabFold/beta/protein.patch")
    pbar.update(4)

    #install biopython
    os.system(f"pip install biopython dm-haiku==0.0.5 ml-collections py3Dmol")
    pbar.update(6)

    # download model params (speedup from kaczmarj)
    os.system(f"mkdir --parents {PARAMS_DIR}")
    os.system(f"curl -fsSL {SOURCE_URL} | tar x -C {PARAMS_DIR}")
    pbar.update(14+27)

    # install hhsuite
    os.system(f"curl -fsSL https://github.com/soedinglab/hh-suite/releases/download/v3.3.0/hhsuite-3.3.0-SSE2-Linux.tar.gz | tar xz -C {TMP_DIR}/")

    os.system('conda install -q -y -c conda-forge openmm pdbfixer mdtraj openmm-plumed openmmforcefields')
    pbar.update(24)

  else:
    pbar.update(total)

###############################################################################################
####    Python imports 
###############################################################################################
if 'alphafold' not in sys.path:
  sys.path.append('alphafold')
if 'ColabFold/beta' not in sys.path:
  sys.path.append('ColabFold/beta')

if f"{TMP_DIR}/bin" not in os.environ['PATH']:
  os.environ['PATH'] += f":{TMP_DIR}/bin:{TMP_DIR}/scripts"

import colabfold as cf
import colabfold_alphafold as cf_af
import mdtraj as md
from openmmplumed import PlumedForce
import json
import matplotlib.pyplot as plt
import numpy as np

try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False
t2 = time.perf_counter()
print(f'time taken to run:{(t2-t1)/60:.2f} mins')
!cd ..

Running on GPU


  0%|          | 0/75 

/usr/local/lib/python3.8/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


time taken to run:8.84 mins


In [ ]:
#@title 1.3) Import libraries
%cd /content/
!rm -r AF2RAVE-toolset
!git clone https://github.com/Akash-Aranganathan/AF2RAVE-toolset.git
!cp AF2RAVE-toolset/*.py .
import plumed_methods as plumed
import rave_methods as rave
import importlib
importlib.reload(plumed)
importlib.reload(rave)

/content
Cloning into 'AF2RAVE-toolset'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 40 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), 21.56 KiB | 1.08 MiB/s, done.


<module 'rave_methods' from '/content/rave_methods.py'>

# 2) Reduced MSA Alphafold2 using Colabfold

In [ ]:
#@title 2.1) Run Colabfold with reduced MSA
#Code was taken from COLABFOLD git and modified for convenience 
import re
import time
from IPython.display import clear_output

%cd /content/struct_gen/.
#####################################################################################################################
###    Input sequence
#####################################################################################################################

#@markdown Change this to the amino acid sequence of your system
sequence = 'MQRGKVKWFNNEKGYGFIEVEGGSDVFVHFTAIQGEGFKTLEEGQEVSFEIVQGNRGPQAANVVKE' #@param {type:"string"}
jobname = "CSP" #@param {type:"string"}
homooligomer =  "1" #param {type:"string"}


#####################################################################################################################
###    MSA parameters
#####################################################################################################################


add_custom_msa = False 
msa_format = "fas" 
pair_mode = "unpaired" 
pair_cov = 50 
pair_qid = 20 

I = cf_af.prep_inputs(sequence, jobname, homooligomer, clean=IN_COLAB)
msa_method = "mmseqs2" 
#I['output_dir']='_'.join(I['output_dir'].split('_')[:-1])
I = cf_af.prep_msa(I, msa_method, add_custom_msa, msa_format,
                   pair_mode, pair_cov, pair_qid, TMP_DIR=TMP_DIR)
mod_I = I
clear_output() # ----> Clear the output from previous functions
#No relaxation 
num_relax = "None"


#####################################################################################################################
###    Parameters for running Alphafold
#####################################################################################################################
rank_by = "pLDDT" 
use_turbo = True 
#@markdown This parameter can be "tuned". Too low, and it doesn't have enough information for sensible predictions. Too high, and it will not generate structural diversity
max_msa = "08:16" #@param ["512:1024", "256:512", "128:256", "64:128", "32:64","16:32","08:16","04:08","02:04","01:02","02:02"]
#@markdown - `max_msa` defines: `max_msa_clusters:max_extra_msa` number of sequences to use. 
max_msa_clusters, max_extra_msa = [int(x) for x in max_msa.split(":")]

show_images = False 

num_models = 5 
use_ptm = True 
num_ensemble = 1 
max_recycles = 1 
is_training = True 
num_samples = 4 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
#@markdown - `num_samples` defines the number of random seed. (For each seed 5 different models are predicted)  
subsample_msa = True 

if not use_ptm and rank_by == "pTMscore":
  print("WARNING: models will be ranked by pLDDT, 'use_ptm' is needed to compute pTMscore")
  rank_by = "pLDDT"

# prep input features
feature_dict = cf_af.prep_feats(mod_I, clean=IN_COLAB)
Ls_plot = feature_dict["Ls"]

# prep model options
opt = {"N":len(feature_dict["msa"]),
       "L":len(feature_dict["residue_index"]),
       "use_ptm":use_ptm,
       "use_turbo":use_turbo,
       "max_recycles":max_recycles,
       "tol":0.0,
       "num_ensemble":num_ensemble,
       "max_msa_clusters":max_msa_clusters,
       "max_extra_msa":max_extra_msa,
       "is_training":is_training}

if use_turbo:
  if "runner" in dir():
    # only recompile if options changed
    runner = cf_af.prep_model_runner(opt, old_runner=runner)
  else:
    runner = cf_af.prep_model_runner(opt)
else:
  runner = None

#####################################################################################################################
###    Run Alphafold with low MSA 
#####################################################################################################################
t1 = time.perf_counter()
outs, model_rank = cf_af.run_alphafold(feature_dict, opt, runner, num_models, num_samples, subsample_msa,
                                       rank_by=rank_by, show_images=show_images)



#####################################################################################################################
###    Output folder
#####################################################################################################################

structures_path='/content/Structures'   #Output_folder
os.makedirs(structures_path)
file_path=os.path.join(structures_path,'file_details.txt')
with open(file_path, "w") as file_details:
  for n,key in enumerate(model_rank):
    copy_line=f'cp {I["output_dir"]}/rank_{n+1}_{key}_unrelaxed.pdb {structures_path}/pred_{n+1}.pdb'
    os.system(copy_line)
    if num_relax !="None":
      if n<num_relax: 
        copy_line2=f'cp {I["output_dir"]}/rank_{n+1}_{key}_relaxed.pdb {structures_path}/pred_{n+1}.pdb'
        os.system(copy_line2)
    line = f"pred_{n+1}.pdb pLDDT:{outs[key]['pLDDT']:.2f}" + f" pTMscore:{outs[key]['pTMscore']:.4f}" if use_ptm else ""
    file_details.write(line+"\n")

#os.system(f'rm -r {I[output_dir]}')   #--> run this line to delete the second copy of structures
%cd /content/
os.system(f'zip -FSr Structures.zip {structures_path}')
t2 = time.perf_counter()
clear_output()
print('\nTime taken to generate:',(t2-t1)/60,' mins')

print(f'The structures can be found in {structures_path}')


Time taken to generate: 1.4455648388833349  mins
The structures can be found in /content/Structures



# 3) Plumed file generation and Clustering

Plumed file will be generated according to the choice Collective Variables(CVs).

An option for dihedrals and distance between C-alpha atoms of specified residues is given.

In [ ]:
#@title 3.1) Analysis of structure and Plumed file generation
generate_plumed=True #@param{type:"boolean"}
#CVs_choice='Chi angles' #@param["Chi angles","Distances","Custom"]
#Residues_of _interest='All' #@param["All","Custom"]
#@markdown If `generate_plumed` is True then mention CVs of interest in the following way (if False provide your plumed file)

#@markdown $\;\:$For all residues $\chi$ angles - `chi:{all}`

#@markdown $\;\:$For all residues specific $\chi$ angles (for example $\chi_1$)- `chi1:{all}`

#@markdown $\;\:$For all residues main chain angles($\phi, psi$) - `phi:{all};psi:{all}`

#@markdown $\;\:$For specific residues fill `{1,2,3}` instead of `{all}`

#@markdown $\;\:$For combinations use `;` to seperate

#@markdown $\;\:$Leave the option blank when not required
CVs_angles='chi:{all};phi:{all}' #@param{type:"string"}


#@markdown $\;\:$For distance between residues (CA - CA) - `dist:{(1,2);(2,3);(5,8)}`


CVs_dist='dist:{(8,10);(8,16)}' #@param{type:"string"}

pdb_list=[f'pred_{i+1}.pdb' for i in range(num_samples*5)]
os.chdir('/content/')
os.mkdir('work_dir')
path_pdb='/content/Structures'
path_work='/content/work_dir'
os.chdir('/content/Structures')

start_bold = "\033[1m"
end_bold = "\033[0;0m"
CVs_list=CVs_angles.split(';')
CVs_interest={}
if generate_plumed:
  for inputs in CVs_list:
    label=inputs.split(':')[0]
    vals=inputs.split(':')[1].strip().strip('{ }').split(',')
    if vals[0] not in ['all','All','ALL']:
      vals=[int(i) for i in vals]
      CVs_interest[label]=vals
    else:
      vals=[0]
      CVs_interest[label]=vals
  if CVs_dist.strip() !='':
    label=CVs_dist.split(':')[0]
    vals=CVs_dist.split(':')[1].strip().strip('{ }').split(';')
    vals_fin=[]
    for val in vals:
      val=val.strip().strip('( )').split(',')
      vals_fin.append((int(val[0]),int(val[1])))
    CVs_interest[label]=vals_fin
  plumed_file='plumed_AF2.dat'
  plumed_out='COLVAR_AF2.dat'
  stride=1
  plumed.generate_plumed('pred_1.pdb',CVs_interest,plumed_file,stride,plumed_out)
else:
  print("\n\n"+start_bold+"Please upload your custom plumed file "+end_bold+"\n")
  uploaded = files.upload()
  plumed_file=list(uploaded.keys())[0]
  plumed_out=plumed.get_value_plumed(plumed_file,'FILE')

inputline=' '.join(pdb_list)
os.system(f'mdconvert {inputline} -o AF2samples.xtc')
os.replace('AF2samples.xtc','../work_dir/AF2samples.xtc')
os.replace(f'{plumed_file}',f'../work_dir/{plumed_file}')
os.chdir(path_work)
os.system(f'plumed driver --plumed {plumed_file} --ixtc AF2samples.xtc')

0

In [ ]:
os.chdir(path_work)
#@title 3.2) Clustering using Regular Space clustering
#@markdown Required parameters for clustering

#@markdown Minimum distances between the cluster centers
min_dist=6.5#@param{type:"number"}
#@markdown Tweak the `min_dist` value to obtain a desired number of clusters

#@markdown Maximum number of cluster centers to be determined 
#@markdown (integer greater than 0 required)
max_centers=10#@param{type:"integer"}

CVs=np.loadtxt(plumed_out)[:,1:]
centers,listindices=rave.RegSpaceClustering(CVs,min_dist,max_centers, batch_size=50) #change to 5 or 5.5 for robustness
print("\n"+start_bold+f"Total number clusters: {len(listindices)}"+end_bold+"\n")
print("The Cluster centers are found at the structures:\n")
for i in listindices:
  print(f"Structure {pdb_list[i-1]} \n")




Total number clusters: 2

The Cluster centers are found at the structures:

Structure pred_1.pdb 

Structure pred_16.pdb 



# 4) Generate  output for unbiased simulation
(Including a plumed file for a specific forcefield)

Note: In case of custom CVs (not listed above), a plumed file has to be generated by the user.

In [ ]:
#@title 4.1) Prepare outputs
generate_plumed_hydrogen=True #@param{type:"boolean"}
#@markdown Choose the `forcefield` for adding hydrogens and creating plumed

forcefield='amber03.xml' #@param['None','amber03.xml','amber14-all.xml','charmm36.xml','amber99sbildn.xml','amoeba2018.xml']
os.chdir(path_work)
os.mkdir('block1_output')

for i in listindices:
  if forcefield!='None':
      init_pdb=pdb_list[i-1]
      os.mkdir(f'{i}')
      os.chdir(f'{i}')
      os.system(f'cp {path_pdb}/{pdb_list[i-1]} .')
      rave.add_hydrogen(i,forcefield)
      os.system(f'cp fixed_Hydrogen_{i}.pdb ../block1_output/.')
      os.chdir(path_work)
  else:
    os.system(f'cp {path_pdb}/{pdb_list[i-1]} {path_work}/block1_output/.')
    os.chdir(path_work)
if generate_plumed:
  plumed_file='plumed_unb.dat'
  plumed_out='COLVAR_unb.dat'
  stride=1000
  os.chdir(f'{path_work}/block1_output/')
  if forcefield!='None':
    plumed.generate_plumed(f'fixed_Hydrogen_{listindices[0]}.pdb',CVs_interest,plumed_file,stride,plumed_out)


We are at /content/work_dir/1
We are at /content/work_dir/16


In [ ]:
#@title 4.2) Download the output
os.chdir(path_work)
os.system(f'zip -r block1_output.zip block1_output')
files.download('block1_output.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>